### Intro to Pydantic

#### Basic Model

In [1]:
from pydantic import BaseModel

In [2]:
class Person(BaseModel):
    first_name: str
    last_name: str
    age: int

In [8]:
p = Person(first_name='Jhon', last_name='Smith', age=24)
p

Person(first_name='Jhon', last_name='Smith', age=24)

In [10]:
p = Person(first_name='Jhon', last_name='Smith', age='24')
p

Person(first_name='Jhon', last_name='Smith', age=24)

In [13]:
from pydantic import ValidationError

try:
    p = Person(first_name='Jhon', last_name=100, age='24kjb')
    p
except Exception as e:
    print(e)

2 validation errors for Person
last_name
  Input should be a valid string [type=string_type, input_value=100, input_type=int]
    For further information visit https://errors.pydantic.dev/2.8/v/string_type
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='24kjb', input_type=str]
    For further information visit https://errors.pydantic.dev/2.8/v/int_parsing


##### Mutation --> While changing the data it will not check for the validations by default

In [14]:
p.age = 'junk'
p

Person(first_name='Jhon', last_name='Smith', age='junk')

#### Validation Exceptions

In [27]:
try:
    p = Person(first_name='Jhon', last_name=100, age='24kjb')
    p
except Exception as e:
    exceptions = e

In [28]:
exceptions.errors()

[{'type': 'string_type',
  'loc': ('last_name',),
  'msg': 'Input should be a valid string',
  'input': 100,
  'url': 'https://errors.pydantic.dev/2.8/v/string_type'},
 {'type': 'int_parsing',
  'loc': ('age',),
  'msg': 'Input should be a valid integer, unable to parse string as an integer',
  'input': '24kjb',
  'url': 'https://errors.pydantic.dev/2.8/v/int_parsing'}]

#### Deserializing the data

In [35]:
# Serializing data from dictionary

data = {
    'first_name' : "Rohit",
    'last_name' : "Kumar",
    'age' : 27,
}

Person.model_validate(data)

Person(first_name='Rohit', last_name='Kumar', age=27)

In [36]:
# Serializing data from JSON object

data = '''
{
    "first_name" : "Rohit",
    "last_name" : "Kumar",
    "age" : 27
}
'''

Person.model_validate_json(data)

Person(first_name='Rohit', last_name='Kumar', age=27)

#### Required vs Optional Fields

In [37]:
try:
    Person(age=27)
except Exception as e:
    print(e)

2 validation errors for Person
first_name
  Field required [type=missing, input_value={'age': 27}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
last_name
  Field required [type=missing, input_value={'age': 27}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing


In [38]:
data = {'age':27}

try:
    Person.model_validate(data)
except Exception as e:
    print(e)

2 validation errors for Person
first_name
  Field required [type=missing, input_value={'age': 27}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing
last_name
  Field required [type=missing, input_value={'age': 27}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing


In [39]:
class Person(BaseModel):
    first_name: str 
    last_name: str
    age: int = 0

In [40]:
Person.model_fields

{'first_name': FieldInfo(annotation=str, required=True),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=False, default=0)}

In [42]:
Person(first_name="Rohit",last_name="Kumar"),Person(first_name="Rohit",last_name="Kumar", age=27) 

(Person(first_name='Rohit', last_name='Kumar', age=0),
 Person(first_name='Rohit', last_name='Kumar', age=27))

#### Nullable Fiedls

In [44]:
class Person(BaseModel):
    # None and String are not of same type
    first_name: str | None = None
    last_name: str
    age: int = 0

In [45]:
Person.model_fields

{'first_name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=False, default=0)}

In [46]:
Person(last_name="Kumar")

Person(first_name=None, last_name='Kumar', age=0)

In [47]:
from typing import Union

class Person(BaseModel):
    first_name: Union[str, None] = None
    last_name: str
    age: int = 0

In [48]:
Person.model_fields

{'first_name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=False, default=0)}

In [50]:
from typing import Optional

class Person(BaseModel):
    first_name: Optional[str]
    last_name: str
    age: int = 0

Person.model_fields

{'first_name': FieldInfo(annotation=Union[str, NoneType], required=True),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=False, default=0)}

In [53]:
class Person(BaseModel):
    first_name: str | None = None
    last_name: str
    age: int = 0
    lucky_numbers: list[int] = []

Person.model_fields

{'first_name': FieldInfo(annotation=Union[str, NoneType], required=False, default=None),
 'last_name': FieldInfo(annotation=str, required=True),
 'age': FieldInfo(annotation=int, required=False, default=0),
 'lucky_numbers': FieldInfo(annotation=list[int], required=False, default=[])}

In [56]:
# Coersion is happening --> Trying to converting the data to specified dataType where possible
p = Person(last_name='Kumar',lucky_numbers=[1,'2',3.0])
p

Person(first_name=None, last_name='Kumar', age=0, lucky_numbers=[1, 2, 3])

In [59]:
for num in p.lucky_numbers:
    print({num : type(num)})

{1: <class 'int'>}
{2: <class 'int'>}
{3: <class 'int'>}


#### Aliased and Field Classes

In [1]:
data = {
    'id': 1,
    'First Name': 'Rohit',
    'LASTNAME': 'Kumar',
    'Age IN YEARS': 27,
}

In [15]:
from pydantic import Field, BaseModel, ValidationError

class Person(BaseModel):
    id_:int = Field(alias='id')
    first_name:str = Field(alias='First Name')
    last_name:str = Field(alias='LASTNAME')
    age:int = Field(alias='Age IN YEARS')

In [6]:
p = Person.model_validate(data)
p

Person(id_=1, first_name='Rohit', last_name='Kumar', age=27)

In [7]:
p.model_dump()

{'id_': 1, 'first_name': 'Rohit', 'last_name': 'Kumar', 'age': 27}

In [8]:
p.model_dump_json()

'{"id_":1,"first_name":"Rohit","last_name":"Kumar","age":27}'

In [9]:
p.model_dump(by_alias=True)

{'id': 1, 'First Name': 'Rohit', 'LASTNAME': 'Kumar', 'Age IN YEARS': 27}

In [10]:
p.model_dump_json(by_alias=True)

'{"id":1,"First Name":"Rohit","LASTNAME":"Kumar","Age IN YEARS":27}'

In [16]:
class Person(BaseModel):
    first_name:str | None = Field(alias='firstName', default=None)
    last_name:str = Field(alias='lastName')

In [17]:
data = {
    'lastName' : 'Kumar'
}

Person.model_validate(data)

Person(first_name=None, last_name='Kumar')

In [20]:
try:
    Person(last_name='Kumar')
except ValidationError as e:
    print(e)

1 validation error for Person
lastName
  Field required [type=missing, input_value={'last_name': 'Kumar'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing


In [21]:
data = {
    'last_name' : 'Kumar' 
}

try:
    Person.model_validate(data)
except ValidationError as e:
    print(e)

1 validation error for Person
lastName
  Field required [type=missing, input_value={'last_name': 'Kumar'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing


#### Model Config: Populate by Name

In [22]:
from pydantic import ConfigDict

class Person(BaseModel):
    model_config = ConfigDict(populate_by_name=True)

    first_name:str | None = Field(alias='firstName', default=None)
    last_name:str = Field(alias='lastName')

In [24]:
p = Person(first_name='Rohit', lastName='Kumar')
p

Person(first_name='Rohit', last_name='Kumar')

In [26]:
data = {
    'first_name' : 'Rohit',
    'lastName' : 'Kumar'
}

Person.model_validate(data)

Person(first_name='Rohit', last_name='Kumar')

#### Mutable Defaults

In [27]:
class Model(BaseModel):
    numbers: list[int] = []

m1 = Model()
m2 = Model()

In [29]:
m1.numbers.extend([1,2,3])
m1

Model(numbers=[1, 2, 3])

In [30]:
m2

Model(numbers=[])

#### Default Factory

In [31]:
from datetime import datetime, timezone

In [34]:
class Log(BaseModel):
    dt : datetime = Field(default_factory=lambda : datetime.now(timezone.utc))
    msg : str

In [38]:
log1 = Log(msg='Hi')
log2 = Log(msg='Hi')
log3 = Log(msg='Hi')

In [39]:
log1.dt, log2.dt, log3.dt

(datetime.datetime(2024, 12, 22, 6, 21, 5, 199942, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 12, 22, 6, 21, 5, 200001, tzinfo=datetime.timezone.utc),
 datetime.datetime(2024, 12, 22, 6, 21, 5, 200039, tzinfo=datetime.timezone.utc))

#### Custom Serializers

In [40]:
class Model(BaseModel):
    dt: datetime

In [41]:
m = Model(dt = datetime.now(timezone.utc))
m

Model(dt=datetime.datetime(2024, 12, 22, 6, 27, 11, 622310, tzinfo=datetime.timezone.utc))

In [42]:
m.model_dump()

{'dt': datetime.datetime(2024, 12, 22, 6, 27, 11, 622310, tzinfo=datetime.timezone.utc)}

In [43]:
m.model_dump_json()

'{"dt":"2024-12-22T06:27:11.622310Z"}'

In [45]:
from pydantic import field_serializer

class Model(BaseModel):
    number: float

    @field_serializer('number')
    def serialize_float(self, value):
        return round(value,2)

In [46]:
m = Model(number=1/3)
m.model_dump()

{'number': 0.33}

In [47]:
m.model_dump_json()

'{"number":0.33}'

In [49]:
class Model(BaseModel):
    number: float
    dt: datetime = Field(default_factory=lambda: datetime.now(timezone.utc))

    @field_serializer('number')
    def serialize_float(self,value):
        return round(value,2)

    @field_serializer('dt', when_used='json-unless-none')
    def serialize_dt_to_json(self, value):
        return value.strftime("%Y/%-m/%-d %I:%M %p")

In [55]:
m = Model(number=1/3)
m

Model(number=0.3333333333333333, dt=datetime.datetime(2024, 12, 22, 6, 38, 41, 496509, tzinfo=datetime.timezone.utc))

In [57]:
m.model_dump()

{'number': 0.33,
 'dt': datetime.datetime(2024, 12, 22, 6, 38, 41, 496509, tzinfo=datetime.timezone.utc)}

In [56]:
m.model_dump_json()

'{"number":0.33,"dt":"2024/12/22 06:38 AM"}'